$\newcommand{\To}{\Rightarrow}$
$\newcommand{\false}{\mathrm{false}}$

In [3]:
import os, sys
sys.path.append(os.path.split(os.getcwd())[0])

In [12]:
from kernel.type import boolT
from kernel.term import Var
from kernel.report import ProofReport
from logic import basic
from logic import matcher
from logic.proofterm import ProofTerm, ProofTermDeriv, ProofTermMacro
from syntax import printer

thy = basic.loadTheory('logic_base')

## Macros

In this section, we introduce the central concept of macros. Macros are deduction rules that serve as abbreviations for more elementary proof steps. In both linear proof and proof terms, one can use an invocation of a macro to substitute for multiple steps of proof. The macro invocations can be expanded on demand during proof checking. Hence, they shorten the length of proofs that need to be stored, and make the whole system scalable to large proofs.

The standard way to create a macro is to inherit from class `ProofTermMacro` (so called because it constructs a macro from functions working with proof terms). The new macro class needs to implement the `get_proof_term` method, which takes as arguments the current theory, arguments to the macro, and a list of input proof terms. If the inputs are valid, the function should return the resulting proof term.

Again, we use applying a theorem as an example, this time implementing it as a macro. We choose the name `apply_theorem_test` to avoid name conflicts with the existing `apply_theorem` macro.

In [5]:
class apply_theorem_test(ProofTermMacro):
    def __init__(self):
        self.level = 1

    def get_proof_term(self, thy, th_name, prevs):
        pt = ProofTerm.theorem(thy, th_name)
        As, _ = pt.prop.strip_implies()  # list of assumptions of pt
        instsp = dict(), dict()      # initial (empty) instantiation
        for A, prev in zip(As, prevs):   # match each assumption with corresponding prev
            matcher.first_order_match_incr(A, prev.prop, instsp)
        tyinst, inst = instsp
        pt2 = ProofTerm.subst_type(tyinst, pt) if tyinst else pt   # perform substitution on pt
        pt3 = ProofTerm.substitution(inst, pt2) if inst else pt2
        for prev in prevs:                   # perform implies_elim on pt
            pt3 = ProofTerm.implies_elim(pt3, prev)
        return pt3

The implementation of `get_proof_term` in `apply_theorem_test` is exactly the same as `apply_theorem` function in the previous section. We can test this macro independently:

In [6]:
A = Var("A", boolT)
B = Var("B", boolT)
ptA = ProofTerm.assume(A)
ptB = ProofTerm.assume(B)
ptAB = apply_theorem_test().get_proof_term(thy, 'conjI', [ptA, ptB])
prf = ptAB.export()
thy.check_proof(prf)
print(printer.print_proof(thy, prf, unicode=True))

0: ⊢ A ⟶ B ⟶ A ∧ B by theorem conjI
1: ⊢ A ⟶ B ⟶ A ∧ B by substitution {A: A, B: B} from 0
2: A ⊢ A by assume A
3: A ⊢ B ⟶ A ∧ B by implies_elim from 1, 2
4: B ⊢ B by assume B
5: A, B ⊢ A ∧ B by implies_elim from 3, 4


To use this macro in proofs, we need to first register it to the theory object. For this, we use the `add_proof_macro` method. It takes two arguments: the name of the macro, and the macro object.

In [7]:
thy.add_proof_macro('apply_theorem_test', apply_theorem_test())

Even though the name of the macro agrees with the class name in this case, this is not required. Nor is it necessary that the macro object be built from a constructor taking no arguments. It can be built in any other way, as long as the `get_proof_term` method is present. This also marks the first time that we modified the theory object directly. As it is now clear, it is the job of the theory object to keep track of the set of macros that can be used in proofs, in addition to the list of types, constants, theorems, and so on.

In [9]:
ptA = ProofTerm.assume(A)
ptB = ProofTerm.assume(B)
ptAB = ProofTermDeriv('apply_theorem_test', thy, 'conjI', prevs=[ptA, ptB])
print(printer.print_thm(thy, ptAB, unicode=True))

A, B ⊢ A ∧ B


As we can see, the sequent obtained by `ptAB` is computed immediately, using the `get_proof_term` function. However, the full proofs is not stored:

In [10]:
prf = ptAB.export()
print(printer.print_proof(thy, prf, unicode=True))

0: assume A
1: assume B
2: apply_theorem_test conjI from 0, 1


In the exported proof, the proof rule `apply_theorem_test` appears directly. Let us now check the proof while keeping the report:

In [15]:
rpt = ProofReport()
thy.check_proof(prf, rpt)
print(printer.print_proof(thy, prf, unicode=True), "\n")
print(rpt)

0: A ⊢ A by assume A
1: B ⊢ B by assume B
2: A, B ⊢ A ∧ B by apply_theorem_test conjI from 0, 1 

Steps: 6
  Theorems:  1
  Primitive: 5
  Macro:     0
Theorems applied: conjI
Macros evaluated: 
Macros expanded: apply_theorem_test
Gaps: []


This time the report is quite informative. While the original proof is only 3 steps, the report says 6 steps were taken, of which there is one application of a theorem (`conjI`) and 5 primitive steps. In addition, the macro `apply_theorem_test` is expanded. This means during proof checking, the full proof behind `apply_theorem_test` is expanded, first using its `get_proof_term` function to obtain a proof term, then converting that proof term into a linear proof.